<a href="https://colab.research.google.com/github/Trailblazer29/Resume-Scanner/blob/master/resume_scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<center>Resume Scanner For Applicant Tracking Systems (ATS)</center>

# Install Required Packages & Software

In [76]:
!pip install docx2txt
!pip install pdfminer
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


# Import Packages

In [77]:
import io
import os
import docx2txt
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
import pytesseract
from PIL import Image
from google.colab import files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Convert any file type (DOCX, PDF, TXT, IMAGE) to text

This function converts any file type to text. The conversion of DOCX files is handled using the **doc2txt** package, PDF files are converted using **PDFMiner** and text is extracted from images using the **Tesseract** OCR engine.

In [78]:
def file_to_text(path):

    file_name, file_extension = os.path.splitext(path)

    if file_extension == ".docx":
        text = docx2txt.process(path)
        return text

    elif file_extension == ".pdf":
      resource_manager = PDFResourceManager()
      file_handler = io.StringIO()
      converter = TextConverter(resource_manager, file_handler)
      page_interpreter = PDFPageInterpreter(resource_manager, converter)
      with open(path, 'rb') as f:
          for page in PDFPage.get_pages(f, caching=True,check_extractable=True):           
              page_interpreter.process_page(page)     
          text = file_handler.getvalue()      
      converter.close() 
      file_handler.close() 
      return text

    elif file_extension == ".txt":
      with open(path, "r") as f:
        text = f.read()
      f.close()
      return text

    else:
      image = Image.open(path)
      text = pytesseract.image_to_string(image)
      return text

# Open Resume & Job Description

Select a job description and a resume and convert them into texts.

In [79]:
job_description = files.upload()
resume = files.upload()

Saving 1.JPG to 1.JPG


Saving 5.JPG to 5.JPG


In [80]:
# Job Description to text
file_name = next(iter(job_description))
path = "/content/" + file_name
job_description = file_to_text(path)

# Resume to text
file_name = next(iter(resume))
path = "/content/" + file_name
resume = file_to_text(path)

content = [job_description, resume]

# Evaluate Similarity between Resume & Job Description

Finally, the similarity between both files will be assessed based on the cosine similarity from **Scikit-learn**.

In [81]:
cv = CountVectorizer()
matrix = cv.fit_transform(content)
similarity_matrix = cosine_similarity(matrix)
similarity = round(similarity_matrix[0][1], 2)*100
print("Resume matches "+str(similarity)+"% of the job description.")

Resume matches 7.000000000000001% of the job description.
